In [84]:
import os
import json
import copy
import pandas as pd
import petl as etl
from pathlib import Path
# from tqdm import tqdm
from tqdm.notebook import tqdm

In [85]:
# TODO: parser zaczynający się od rozpakowania zipa
ROOT_DIR = "./dataset/"

In [86]:
def crawler(root_dir: str = ROOT_DIR) -> None:
    ALL_COLS = set()
    for root, dirs, files in tqdm(os.walk(root_dir), desc="Crawling Dataset"):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    json_data = json.load(f)
                    for offer in json_data:   
                        data = degerate_data(json_data)
                        del json_data
                        return data


In [87]:
columns_og = ['latitude', 'multilocation', 'longitude', 'remote', 'address_text', 'company_logo_url', 'published_at', 'street', 'company_name', 'skills', 'marker_icon', 'display_offer', 'country_code', 'way_of_apply', 'experience_level', 'id', 'employment_types', 'remote_interview', 'company_url', 'company_size', 'workplace_type', 'open_to_hire_ukrainians', 'title', 'city']

columns_to_check = ('latitude', 'multilocation', 'longitude', 'remote', 'address_text', 'published_at', 'street', 'company_name', 'skills', 'marker_icon', 'display_offer', 'country_code', 'way_of_apply', 'experience_level', 'id', 'remote_interview', 'company_url', 'company_size', 'workplace_type', 'open_to_hire_ukrainians', 'title', 'city')

In [88]:
def degerate_data(data):
    tmp = []
    # zdegenerowane dane
    # see every offer
    for offer in data:
        offer_cp = copy.copy(offer)
        del offer_cp["company_logo_url"]
        del offer_cp["employment_types"]
        # cross-check to ensure that all columns all present even with None
        
        cols_to_add = set(columns_to_check) - set(offer_cp.keys())
        
        for col in cols_to_add:
            offer_cp[col] = None
        
        for empl_types in offer["employment_types"]:
            offer_cp["employment_type"] = empl_types["type"]
            offer_cp["salary"] = "Disclosed"
            if empl_types["salary"] is not None:
                offer_cp["from"] = empl_types["salary"]["from"]
                offer_cp["to"] = empl_types["salary"]["to"]
                offer_cp["currency"] = empl_types["salary"]["currency"]
            else:
                offer_cp["from"] = None 
                offer_cp["to"] = None
                offer_cp["currency"] = None
               
                tmp.append(offer_cp)
    return tmp
            


In [89]:
columns = ['latitude', 'multilocation', 'longitude', 'remote', 'address_text', 'company_logo_url', 'published_at', 'street', 'company_name', 'skills', 'marker_icon', 'display_offer', 'country_code', 'way_of_apply', 'experience_level', 'id', 'employment_types', 'remote_interview', 'company_url', 'company_size', 'workplace_type', 'open_to_hire_ukrainians', 'title', 'city']

In [90]:
jobs = []
data = crawler(ROOT_DIR)


Crawling Dataset: 0it [00:00, ?it/s]

In [91]:
data = pd.DataFrame(data)
print(data.head())

                          title          street      city country_code  \
0               DevOps Engineer      Dzielna 60  Warszawa           PL   
1               DevOps Engineer      Dzielna 60  Warszawa           PL   
2  MES Developer systemu Apriso  Wólczańska 178      Łódź           PL   
3             Software Engineer               -   Wrocław           PL   
4             Software Engineer               -    Kraków           PL   

           address_text marker_icon workplace_type  \
0  Dzielna 60, Warszawa      devops         remote   
1  Dzielna 60, Warszawa      devops         remote   
2  Wólczańska 178, Łódź  javascript         remote   
3            -, Wrocław         net         remote   
4             -, Kraków         net         remote   

                 company_name                      company_url company_size  \
0  Transition Technologies MS               http://www.ttms.pl         500+   
1  Transition Technologies MS               http://www.ttms.pl         5

In [92]:
def create_dims_tables(data, cols: str|dict):
    # colname lub {"name": [cola, colb, colc]}
    if type(cols) == str:
        dimname = cols + "_id"
    else:
        dimname = cols.keys()[0] +" _id"
        cols = cols[dimname]
    
    data_dim = data[cols].drop_duplicates().reset_index(drop=True)
    data_dim[dimname] = dim_company.index + 1
    data.merge(data_dim, on=dimname)
    return data, data_dim
    

In [93]:
df = data
# Tworzenie tabeli wymiarów dla firmy
dim_company = df[['company_name', 'company_url', 'company_size']].drop_duplicates().reset_index(drop=True)
dim_company['company_id'] = dim_company.index + 1

# Tworzenie tabeli wymiarów dla lokalizacji
dim_location = df[['street', 'city', 'country_code', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
dim_location['location_id'] = dim_location.index + 1

# Tworzenie tabeli wymiarów dla umiejętności
# Rozwijanie listy umiejętności na osobne wiersze
skills_expanded = df.explode('skills')[['id', 'skills']]
skills_expanded = pd.concat([skills_expanded.drop(['skills'], axis=1), skills_expanded['skills'].apply(pd.Series)], axis=1)
skills_expanded = skills_expanded.rename(columns={'name': 'skill_name', 'level': 'skill_level'}).drop_duplicates().reset_index(drop=True)

# Tworzenie tabeli wymiarów dla umiejętności
dim_skills = skills_expanded[['skill_name', 'skill_level']].drop_duplicates().reset_index(drop=True)
dim_skills['skill_id'] = dim_skills.index + 1

# Dodanie identyfikatorów umiejętności do tabeli umiejętności rozszerzonej
skills_expanded = skills_expanded.merge(dim_skills, on=['skill_name', 'skill_level'])

# Grupowanie umiejętności według id ofert pracy
skills_grouped = skills_expanded.groupby('id')['skill_id'].apply(list).reset_index()

# Łączenie umiejętności z tabelą faktów
df = df.merge(skills_grouped, on='id')

# Dołączenie identyfikatorów firmy i lokalizacji do tabeli faktów
df = df.merge(dim_company, on=['company_name', 'company_url', 'company_size'])
df = df.merge(dim_location, on=['street', 'city', 'country_code', 'latitude', 'longitude'])

# Przekształcenie tabeli faktów, aby zawierała tylko identyfikatory oraz inne istotne kolumny
fact_df = df[['id', 'title', 'workplace_type', 'experience_level', 'published_at', 'remote_interview', 'open_to_hire_ukrainians', 'remote', 'employment_type', 'salary', 'from', 'to', 'currency','company_id', 'location_id', 'skill_id']]

In [94]:
dim_location

,street,city,country_code,latitude,longitude,location_id
0,Dzielna 60,Warszawa,PL,52.2437256,20.9833704,1
1,Wólczańska 178,Łódź,PL,51.7539054,19.456101,2
2,-,Wrocław,PL,51.1078852,17.0385376,3
3,-,Kraków,PL,50.0646501,19.9449799,4
4,Inflancka 4a,Warszawa,PL,52.2564721,20.993809,5
...,...,...,...,...,...,...
745,Kalifornia,Cupertino,US,37.3229978,-122.0321823,746
746,Strzelców Bytomskich 87,Bytom,PL,50.371158,18.893748,747
747,Rydygiera 8,Warszawa,PL,52.2296756,21.0122287,748
748,Reymonta 13,Wrocław,PL,51.129771,17.0279918,749


In [95]:
print(fact_df.head())

                                                  id  \
0         transition-technologies-ms-devops-engineer   
1         transition-technologies-ms-devops-engineer   
2        accenture-mes-developer-systemu-apriso-lodz   
3                     ifs-software-engineer-warszawa   
4  ifs-software-engineer-a363af06-8ee7-4934-a9dc-...   

                          title workplace_type experience_level  \
0               DevOps Engineer         remote              mid   
1               DevOps Engineer         remote              mid   
2  MES Developer systemu Apriso         remote              mid   
3             Software Engineer         remote              mid   
4             Software Engineer         remote              mid   

               published_at  remote_interview  open_to_hire_ukrainians  \
0  2022-02-28T16:28:00.000Z              True                    False   
1  2022-02-28T16:28:00.000Z              True                    False   
2  2022-02-28T16:00:15.631Z           

In [96]:
fact_df.shape

(2686, 16)